# 🐸 EZ-FRCNN Demo: First-Time Use with Preloaded Frog Dataset

Welcome to the EZ-FRCNN demo notebook! This walkthrough will guide you through:

- ✅ Training an object detection model on a **preloaded frog dataset**
- ✅ Running inference on a held-out test set
- ✅ Visualizing and exporting detection results

This notebook is built for **first-time users** — no setup or data prep is required beyond running the cells.

---

## 📁 Preloaded Dataset

The demo dataset is already included and organized as follows:

- `images/train` — training images and annotations  
- `images/test` — test images and annotations

> ✅ No need to modify or download anything. The data is ready to use.

---

## 🚀 How to Use This Notebook

1. Run all cells in order (top to bottom)
2. The model will train on `images/train`and validate on `images/test`
3. Inference will run on `test_data/test_images`
4. Detections will be visualized and saved

This demo confirms your EZ-FRCNN installation works and gives you a complete example pipeline.

---

## 💡 Try EZ-FRCNN on Your Own Images

Once the demo completes, you can run inference on your own images using the `ezfrcnn.ipynb` notebook


In [ ]:
from library import *
from tqdm.auto import tqdm

Training parameters are defined below, no need to change these on the first run

In [ ]:
BATCH_SIZE = 4 # increase / decrease according to GPU memory
RESIZE_TO = 512 # resize the image for training and transforms
NUM_EPOCHS = 20 # number of epochs to train for
SAVE_PLOTS_EPOCH = 1 # save loss plots after these many epochs
SAVE_MODEL_EPOCH = 5 # save model after these many epochs

Run the cell below to load in your images and annotations. This cell will print the number of images found in the testing and training folder

In [ ]:
# prepare the final datasets and data loaders
train_dataset = getDataset(TRAIN_DIR, RESIZE_TO, RESIZE_TO, get_train_transform())
valid_dataset = getDataset(VALID_DIR, RESIZE_TO, RESIZE_TO, get_valid_transform())
[train_loader, valid_loader] = get_loaders(train_dataset, valid_dataset, BATCH_SIZE, collate_fn)

NUM_CLASSES = len(train_dataset.classes)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}\n")

Run the cell below to visualize your data to ensure that bounding boxes match images as expected. There will be gridlines in the image shown

In [ ]:
INDEX_TO_VIS = 0
print(train_dataset.classes)
visualize_sample(TRAIN_DIR, RESIZE_TO, INDEX_TO_VIS)

To start training from the default COCO weights run the cell below

In [ ]:
model = create_model(num_classes=NUM_CLASSES)

Train your model by running the cell below! After the specified number of epochs passes, the model and a plot of the training/validation loss over time will be saved under `/models` and `/plots` respectively. This should take around 10 minutes on a device with a GPU

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# name to save the trained model with
MODEL_NAME = 'model'
tq1 = tqdm(range(0,NUM_EPOCHS))
fig = plt.figure
[train_loss_list, val_loss_list] = train_model(model, train_loader, valid_loader,
                                               DEVICE, MODEL_NAME, NUM_EPOCHS,
                                               MODEL_DIR, PLOT_DIR, SAVE_MODEL_EPOCH,
                                               SAVE_PLOTS_EPOCH, tq1, fig)
# Plot Validation & Training Loss
plt.plot(train_loss_list)
plt.plot(val_loss_list)

<h1> Inferencing</h1>
Run the cells below to create bounding boxes for new data. Images with bounding boxes overlaid will be present in the outputs folder following inferencing as well. CSV files will also be created, one for the class of each object detected and one for the bounding boxes of said objects. Each row of the CSV file will correspond to each frame of the movie inferenced (in the case of movie inferencing) or each image in the ordering defined by the output images (in the case of image inferencing). Each bounding box is of the form [xmin ymin xmax ymax]
<br> <br>
First define the confidence threshold you would like to use for inferencing

In [ ]:
detection_threshold = 0.9# 0.9 by default

Load your model by running the cell below

In [ ]:
model_name = 'model20.pth'
model = load_model(model_name, MODEL_DIR, NUM_CLASSES)

Finally, run inferencing here: images with bounding boxes overlaid and a csv of the detections will be output in `/outputs`

In [ ]:
# Path to folder of images for inferencing
folderName = './test_data/test_images/'
[boxFileName, classFileName, scoreFileName] = ['boxes', 'classes', 'scores']
inf_fig = plt.figure()
results = inference_images(folderName, model, OUT_DIR, detection_threshold, train_dataset.classes, tqdm, inf_fig)
